# Zenodo API code for getting GitHub Repos 

### Setup  

Ensure you are in the `coding-smart-github` conda environment and have the following python packages in your environment:
  * requests    

#### Github Authentication

Create an access token via Zenodo Applications settings [here](https://zenodo.org/account/settings/applications/tokens/) and create a file called `zenodoconfig.cfg` with the following content:

    [ACCESS]
    token = <your-access-token>
    
Ensure you've pasted in your token, but leave `[ACCESS]` and `token =` .

In [22]:
import pandas as pd
#import numpy as np 
import os 
from os import path
import configparser 
import requests
import csv 
import json

In [2]:
import requests
r = requests.get("https://zenodo.org/api/deposit/depositions")
r.status_code
# 403
r.json()
# permission denied is OK because we didn't do any access token things yet I think 

{'status': 403, 'message': 'Permission denied.'}

In [3]:
# set up zenodo access token

config = configparser.ConfigParser()
config.read('../zenodoconfig.cfg')
config.sections()

access_token = config['ACCESS']['token']

r = requests.get('https://zenodo.org/api/deposit/depositions',
                  params={'access_token': access_token})

print(r)

r.status_code
# 200

r.json()
# []

#r.status_code == requests.codes.ok
#r.raise_for_status()

<Response [200]>


[{'created': '2015-06-25T15:04:19+00:00',
  'modified': '2020-01-24T19:27:28.405139+00:00',
  'id': 12271,
  'conceptrecid': '592847',
  'doi': '10.5281/zenodo.12271',
  'conceptdoi': '',
  'doi_url': 'https://doi.org/10.5281/zenodo.12271',
  'metadata': {'title': 'peramagroon: seed',
   'doi': '10.5281/zenodo.12271',
   'publication_date': '2014-10-16',
   'description': '<p>This is the beginning of my reproducible research / open data / science journey.</p>\n\n<p>I&#39;ll use this repo to document my progress in analysing and publishing results from data collected during fieldwork carried out during my masters (MSc) thesis. The original thesis was on the vegetation of Peramagroon Mountain in Iraqi Kurdistan, and was completed in August 2013.</p>\n\n<p>It will contain my data, data cleaning/tidying and analysis scripts, figures, and manuscripts. I&#39;m also aiming to make everything reproducible and openly accessible, so I&#39;ll also detail these factors and include notes on how I d

In [45]:
# records_api_url = 'https://zenodo.org/api/records'
# search_query = 'type:software AND creators.affiliation:Cambridge'
# res = requests.get(records_api_url, params={'q': search_query, 'size': 25, 'page': 1})

In [4]:
requests.get("https://zenodo.org/api/records/{}".format(8412005)).json()['metadata']['related_identifiers']

[{'identifier': 'https://github.com/sequana/versionix/tree/v0.2.1',
  'relation': 'isSupplementTo',
  'scheme': 'url'}]

In [75]:
records_api_url = 'https://zenodo.org/api/records'
search_query = 'type:software'
r = requests.get(records_api_url, params={'q': search_query, 'size':20})

print(r.status_code)
print(r.links)

200
{}


In [118]:
records_api_url = 'https://zenodo.org/api/records'
search_query = 'type:software'
r = requests.get(records_api_url, params={'q': search_query, 'all_versions': 'true', 'size': 20, 'page': 1})

print(r.status_code)
print(r.links)
print(r.next)
data = r.json()
print(data)

200
{}
None
[{'created': '2021-11-13T14:27:12.158730+00:00', 'modified': '2021-11-14T13:49:23.519181+00:00', 'id': 5694496, 'conceptrecid': '5694495', 'doi': '10.5281/zenodo.5694496', 'conceptdoi': '10.5281/zenodo.5694495', 'doi_url': 'https://doi.org/10.5281/zenodo.5694496', 'metadata': {'title': 'spawaskar-cora/cora-docs: v2.1.0 Open Community Release', 'doi': '10.5281/zenodo.5694496', 'publication_date': '2021-11-13', 'description': '<p>This release includes the following</p>\n<p>The main reason for this release was to tie this to a Zenodo DOI number for citation purposes.\nGIF images for most of the important screens and workflows\nIncludes all available Anomalies, Pathologies, Traumas, and Taphonomies.\nAdded documentation for CoRA Data Export and Import (of measurements for Osteometric Sorting Algorithms).</p>', 'access_right': 'open', 'creators': [{'name': 'Sachin Pawaskar', 'affiliation': None}, {'name': 'Emily Streetman', 'affiliation': None}, {'name': 'C. LeGarde', 'affiliati

In [119]:
records_api_url = 'https://zenodo.org/api/records'
search_query = 'type:software'
r = requests.get(records_api_url, params={'q': search_query, 'all_versions': 'true', 'size': 20, 'page': 2})

print(r.status_code)
print(r.links)
print(r.next)
data = r.json()
print(data)

200
{}
None
[{'created': '2021-01-22T19:52:11.886069+00:00', 'modified': '2021-01-23T00:27:22.757516+00:00', 'id': 4458287, 'conceptrecid': '4458286', 'doi': '10.5281/zenodo.4458287', 'conceptdoi': '10.5281/zenodo.4458286', 'doi_url': 'https://doi.org/10.5281/zenodo.4458287', 'metadata': {'title': 'zadkhosh/intelligentRoutingFramework: ThesisCodes', 'doi': '10.5281/zenodo.4458287', 'publication_date': '2021-01-22', 'description': 'PHDThesis', 'access_right': 'open', 'creators': [{'name': 'zadkhosh', 'affiliation': None}], 'related_identifiers': [{'identifier': 'https://github.com/zadkhosh/intelligentRoutingFramework/tree/v0.1-thesis', 'relation': 'isSupplementTo', 'scheme': 'url'}], 'version': 'v0.1-thesis', 'license': 'other-open', 'imprint_publisher': 'Zenodo', 'upload_type': 'software', 'prereserve_doi': {'doi': '10.5281/zenodo.4458287', 'recid': 4458287}}, 'title': 'zadkhosh/intelligentRoutingFramework: ThesisCodes', 'links': {'self': 'https://zenodo.org/api/records/4458287', 'self

In [99]:
len(r.json())

25

TypeError: the JSON object must be str, bytes or bytearray, not list

In [93]:
dir(r.next)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [6]:
r.raise_for_status()

identifiers = [] 

for record_id in r.json():
    identifiers.append(record_id['id'])



# still_iterating = True

# while (len(identifiers) < 50): 
#     for record_id in r.json()['id']:
#         identifiers.append(record_id)

print(identifiers)

####### read this: https://github.com/dvolgyes/zenodo_get/blob/master/zenodo_get/zget.py  line ~307



[5694496, 5520494, 6500649, 6029408, 6558631, 7495452, 5901450, 5228707, 5946593, 7057132, 6578814, 6700714, 4388632, 6567232, 6162606, 7882172, 6636328, 6783138, 7686292, 6401535]


In [7]:
#for record_id in identifiers: 
#    print(r.json()['metadata']['related_identifiers'])

In [69]:
#### WORK ON THIS - it doesn't work!! 

# f = open('../../data/zenodo-results.csv', 'w')
# writer = csv.writer(f)

# header = ['Zenodo ID', 'Title', 'DOI', 'GitHub Related Link', 'Relation Type']
# writer.writerow(header)

for record_id in identifiers:
    r = requests.get("https://zenodo.org/api/records/{}".format(record_id))

    #if ("github.com" in r.json()['metadata']['related_identifiers']) & ("url" in r.json()['metadata']['related_identifiers']):
    #if ("github.com" in r.json()['metadata']['related_identifiers']):
    if 'metadata' in r.json():
    
        record_title = r.json()['title']
        record_doi = r.json()['doi']
        record_metadata = r.json()['metadata']
        
        if 'related_identifiers' in record_metadata: 
            record_metadata_identifiers = r.json()['metadata']['related_identifiers']
    
            print(
                record_id, 
                record_title, 
                type(record_metadata), 
                type(record_metadata_identifiers), 
                record_metadata_identifiers[0]['identifier']
            )
            
            if ("github.com" in record_metadata_identifiers[0]['identifier']) & ("url" in record_metadata_identifiers[0]['scheme']):
                record_gh_repo_url = record_metadata_identifiers[0]['identifier']
                print(record_gh_repo_url)
        
        else: 
            print(record_id, record_title)

    else: 
        print("no metadata")
        continue
        

    #for id in r.json()['metadata']['related_identifiers']:
    #    if ("github.com" in id['identifier']) & ("url" in id['scheme']):
    #        row = []
    #        row.append(record_id)
    #        row.append(record_title)
    #        row.append(record_doi)
    #        row.append(id['identifier'])
    #        row.append(id['relation'])
    #        writer.writerow(row)

#f.close()

5694496 spawaskar-cora/cora-docs: v2.1.0 Open Community Release <class 'dict'> <class 'list'> https://github.com/spawaskar-cora/cora-docs/tree/v2.1.0
https://github.com/spawaskar-cora/cora-docs/tree/v2.1.0
5520494 sbernasek/binding: Release for P/Y Paper <class 'dict'> <class 'list'> https://github.com/sbernasek/binding/tree/v0.3
https://github.com/sbernasek/binding/tree/v0.3
6500649 Jean-Baptiste-Camps/word_normalisation_data: v0.1 DH Tokyo Abstract <class 'dict'> <class 'list'> https://github.com/Jean-Baptiste-Camps/word_normalisation_data/tree/v0.1
https://github.com/Jean-Baptiste-Camps/word_normalisation_data/tree/v0.1
6029408 pkeys/WAM2layersPython: <class 'dict'> <class 'list'> https://github.com/pkeys/WAM2layersPython/tree/v2.4.08
https://github.com/pkeys/WAM2layersPython/tree/v2.4.08
6558631 bc/stfeasibility: May 17 Submission to PNAS <class 'dict'> <class 'list'> https://github.com/bc/stfeasibility/tree/Pub
https://github.com/bc/stfeasibility/tree/Pub
7495452 TerraPIN: Terrace

In [70]:
# 

f = open('../../data/zenodo-results.csv', 'w')
writer = csv.writer(f)

header = ['Zenodo ID', 'Title', 'DOI', 'GitHub Link']
writer.writerow(header)

for record_id in identifiers:
    r = requests.get("https://zenodo.org/api/records/{}".format(record_id))

    if 'metadata' in r.json():
    
        record_title = r.json()['title']
        record_doi = r.json()['doi']
        record_metadata = r.json()['metadata']
        
        if 'related_identifiers' in record_metadata: 
            record_metadata_identifiers = r.json()['metadata']['related_identifiers']
    
#             print(
#                 record_id, 
#                 record_title, 
#                 type(record_metadata), 
#                 type(record_metadata_identifiers), 
#                 record_metadata_identifiers[0]['identifier']
#             )
            
            if ("github.com" in record_metadata_identifiers[0]['identifier']) & ("url" in record_metadata_identifiers[0]['scheme']):
                
                record_gh_repo_url = record_metadata_identifiers[0]['identifier']
                
                row = []
                row.append(record_id)
                row.append(record_title)
                row.append(record_doi)
                row.append(record_gh_repo_url)
                writer.writerow(row)

                
#                 print(record_gh_repo_url)
        
        else: 
            #print(record_id, record_title)
            continue

    else: 
        #print("no metadata")
        continue

f.close()

In [125]:
records_api_url = 'https://zenodo.org/api/records'
search_query = 'type:software'

page_iterator = 1
identifiers = [] 

while (len(identifiers) < 100):
    
    r = requests.get(records_api_url, params={'q': search_query, 'all_versions': 'true', 'size': 20, 'page': page_iterator})

    r.raise_for_status()
    
    for record_id in r.json():
        identifiers.append(record_id['id'])
        #print(identifiers)
        
        print(len(identifiers))
        
        page_iterator += 1

print(identifiers)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
[5694496, 5520494, 6500649, 6029408, 6558631, 7495452, 5901450, 5228707, 5946593, 7057132, 6578814, 6700714, 4388632, 6567232, 6162606, 7882172, 6636328, 6783138, 7686292, 6401535, 8243106, 7439127, 8125737, 7774907, 7792055, 401135, 1010793, 3964240, 5570390, 5717384, 5873595, 5181822, 4541874, 4633135, 4430074, 4919431, 5090764, 5574462, 6635701, 5420862, 7656586, 8132067, 8180947, 832982, 1298444, 1208675, 6384727, 7595224, 8385388, 8412672, 206353, 854651, 6467636, 7352528, 7613133, 7735184, 7779067, 7792223, 8280714, 8408343, 5654741, 5787688, 5573141, 5721504, 6606972, 6259048, 4769851, 5709970, 7096607, 5526788, 6632129, 7084978, 4756246, 5213063, 6717314, 7063211, 5013872, 5914078, 5975221, 

In [ ]:
### THIS WORKS NICELY 


records_api_url = 'https://zenodo.org/api/records'
search_query = 'type:software'

page_iterator = 1
identifiers = [] 

while (len(identifiers) < 100):
    
    r = requests.get(records_api_url, params={'q': search_query, 'all_versions': 'true', 'size': 20, 'page': page_iterator})

    r.raise_for_status()
    
    for record_id in r.json():
        identifiers.append(record_id['id'])
        #print(identifiers)
        
        print(len(identifiers))
        
        page_iterator += 1

print(identifiers)

In [126]:
#### THIS ALSO WORKS NICELY!!! 


f = open('../../data/zenodo-results.csv', 'w')
writer = csv.writer(f)

header = ['Zenodo ID', 'Title', 'DOI', 'GitHub Link']
writer.writerow(header)

for record_id in identifiers:
    r = requests.get(f"{records_api_url}/{record_id}")

    if 'metadata' in r.json():
    
        record_title = r.json()['title']
        record_doi = r.json()['doi']
        record_metadata = r.json()['metadata']
        
        if 'related_identifiers' in record_metadata: 
            record_metadata_identifiers = r.json()['metadata']['related_identifiers']
    
#             print(
#                 record_id, 
#                 record_title, 
#                 type(record_metadata), 
#                 type(record_metadata_identifiers), 
#                 record_metadata_identifiers[0]['identifier']
#             )
            
            if ("github.com" in record_metadata_identifiers[0]['identifier']) & ("url" in record_metadata_identifiers[0]['scheme']):
                
                record_gh_repo_url = record_metadata_identifiers[0]['identifier']
                
                row = []
                row.append(record_id)
                row.append(record_title)
                row.append(record_doi)
                row.append(record_gh_repo_url)
                writer.writerow(row)

                
#                 print(record_gh_repo_url)
        
        else: 
            #print(record_id, record_title)
            continue

    else: 
        #print("no metadata")
        continue

f.close()

In [26]:
#res.json()[0]['metadata']['related_identifiers']

[{'identifier': 'https://github.com/ultralytics/yolov5/tree/v7.0',
  'relation': 'isSupplementTo',
  'scheme': 'url'}]

In [ ]:
# Field details here: https://help.zenodo.org/guides/search/  

In [96]:
#res.json()[0].get('metadata').keys()

dict_keys(['title', 'doi', 'publication_date', 'description', 'access_right', 'creators', 'related_identifiers', 'version', 'license', 'imprint_publisher', 'upload_type', 'prereserve_doi'])

In [107]:
#res.json()[0].get('metadata').get('related_identifiers')

[{'identifier': 'https://github.com/ultralytics/yolov5/tree/v7.0',
  'relation': 'isSupplementTo',
  'scheme': 'url'}]

In [258]:
content = pd.DataFrame.from_dict(res.json())
 # pulling the json query output into a dataframe 
    # can then begin to pull data out of metadata column... 
    # ... where it's got related_identifiers including github.com

In [259]:
print(content)

                            created                          modified  \
0  2022-11-22T15:29:21.125713+00:00  2022-11-23T02:26:37.566894+00:00   
1  2023-09-15T14:02:38.609718+00:00  2023-09-15T14:26:57.370604+00:00   
2  2023-10-05T08:49:40.550242+00:00  2023-10-05T14:27:13.161473+00:00   
3  2023-09-29T19:01:55.671486+00:00  2023-09-30T02:27:05.766147+00:00   
4  2023-08-26T10:02:01.165552+00:00  2023-08-27T02:26:55.890297+00:00   
5  2023-10-09T04:42:46.029552+00:00  2023-10-09T14:27:00.049603+00:00   
6  2023-09-20T21:02:10.942443+00:00  2023-09-21T02:27:04.290583+00:00   
7  2020-03-20T13:53:21.347110+00:00  2020-03-20T20:20:39.657140+00:00   
8  2023-09-15T02:33:47.463107+00:00  2023-09-15T14:26:56.948563+00:00   
9         2015-07-16T12:45:24+00:00  2020-01-29T13:09:11.520772+00:00   

        id conceptrecid                     doi              conceptdoi  \
0  7347926      3908559  10.5281/zenodo.7347926  10.5281/zenodo.3908559   
1  8349181       596518  10.5281/zenodo.834918

In [260]:
content.links[2]

{'self': 'https://zenodo.org/api/records/8409685',
 'self_html': 'https://zenodo.org/records/8409685',
 'self_doi': 'https://zenodo.org/doi/10.5281/zenodo.8409685',
 'doi': 'https://doi.org/10.5281/zenodo.8409685',
 'parent': 'https://zenodo.org/api/records/1243862',
 'parent_html': 'https://zenodo.org/records/1243862',
 'parent_doi': 'https://zenodo.org/doi/10.5281/zenodo.1243862',
 'self_iiif_manifest': 'https://zenodo.org/api/iiif/record:8409685/manifest',
 'self_iiif_sequence': 'https://zenodo.org/api/iiif/record:8409685/sequence/default',
 'files': 'https://zenodo.org/api/records/8409685/files',
 'media_files': 'https://zenodo.org/api/records/8409685/media-files',
 'archive': 'https://zenodo.org/api/records/8409685/files-archive',
 'archive_media': 'https://zenodo.org/api/records/8409685/media-files-archive',
 'latest': 'https://zenodo.org/api/records/8409685/versions/latest',
 'latest_html': 'https://zenodo.org/records/8409685/latest',
 'draft': 'https://zenodo.org/api/records/84

In [149]:
#### START HERE #####

#### continue by pulling the github.com/*/* content out into a file. 


In [80]:
content.keys()

Index(['created', 'modified', 'id', 'conceptrecid', 'doi', 'conceptdoi',
       'doi_url', 'metadata', 'title', 'links', 'record_id', 'owner', 'files',
       'state', 'submitted'],
      dtype='object')

In [112]:
content['related_identifiers'] = content[['metadata']].apply(lambda x: [x.get('related_identifiers') for x in x])

In [113]:
content

,created,modified,id,conceptrecid,doi,conceptdoi,doi_url,metadata,title,links,record_id,owner,files,state,submitted,related_identifiers,related_identifiers_url
0,2022-11-22T15:29:21.125713+00:00,2022-11-23T02:26:37.566894+00:00,7347926,3908559,10.5281/zenodo.7347926,10.5281/zenodo.3908559,https://doi.org/10.5281/zenodo.7347926,{'title': 'ultralytics/yolov5: v7.0 - YOLOv5 S...,ultralytics/yolov5: v7.0 - YOLOv5 SOTA Realtim...,{'self': 'https://zenodo.org/api/records/73479...,7347926,64193,[{'id': '6d9a4918-34ef-47ed-a472-e11bb6edcdcf'...,done,True,[{'identifier': 'https://github.com/ultralytic...,None
1,2023-09-15T14:02:38.609718+00:00,2023-09-15T14:26:57.370604+00:00,8349181,596518,10.5281/zenodo.8349181,10.5281/zenodo.596518,https://doi.org/10.5281/zenodo.8349181,"{'title': 'python-pillow/Pillow: 10.0.1', 'doi...",python-pillow/Pillow: 10.0.1,{'self': 'https://zenodo.org/api/records/83491...,8349181,17549,[{'id': '453c7390-aa5c-4ca8-b727-d0f11c837828'...,done,True,[{'identifier': 'https://github.com/python-pil...,None
2,2023-10-05T08:49:40.550242+00:00,2023-10-05T14:27:13.161473+00:00,8409685,1243862,10.5281/zenodo.8409685,10.5281/zenodo.1243862,https://doi.org/10.5281/zenodo.8409685,"{'title': 'Scientific colour maps', 'doi': '10...",Scientific colour maps,{'self': 'https://zenodo.org/api/records/84096...,8409685,43498,[{'id': 'df9ec0f1-6ad8-46d2-8485-5e93eb5d8593'...,done,True,"[{'identifier': '10.5194/gmd-2017-328', 'relat...",None


In [243]:
for id in content['related_identifiers']:

    print(id)

[{'identifier': 'https://github.com/ultralytics/yolov5/tree/v7.0', 'relation': 'isSupplementTo', 'scheme': 'url'}]
[{'identifier': 'https://github.com/python-pillow/Pillow/tree/10.0.1', 'relation': 'isSupplementTo', 'scheme': 'url'}]
[{'identifier': '10.5194/gmd-2017-328', 'relation': 'isDocumentedBy', 'scheme': 'doi'}]


In [236]:
for id in content['related_identifiers']:
#         if ("github.com" in id['identifier']):
#             row = []
#             row.append(record_id)
#             row.append(id['identifier'])
#             row.append(id['relation'])

    print(id[0].get('identifier'))
    print(id[0].get('relation'))
    print(id[0].get('scheme'))

https://github.com/ultralytics/yolov5/tree/v7.0
isSupplementTo
url
https://github.com/python-pillow/Pillow/tree/10.0.1
isSupplementTo
url
10.5194/gmd-2017-328
isDocumentedBy
doi


In [238]:
for id in content['related_identifiers']:

    if ("github.com" in id[0].get('identifier')):
        print(id[0].get('identifier'))
        print(id[0].get('relation'))
        print(id[0].get('scheme'))

https://github.com/ultralytics/yolov5/tree/v7.0
isSupplementTo
url
https://github.com/python-pillow/Pillow/tree/10.0.1
isSupplementTo
url


In [239]:
identifiers

0    7347926
1    8349181
2    8409685
Name: id, dtype: int64

In [232]:
content.columns

Index(['created', 'modified', 'id', 'conceptrecid', 'doi', 'conceptdoi',
       'doi_url', 'metadata', 'title', 'links', 'record_id', 'owner', 'files',
       'state', 'submitted', 'related_identifiers', 'related_identifiers_url',
       'related_identifiers_gh', 'related_identifiers_gh_str'],
      dtype='object')

In [22]:
# get zenodo records with type software  
# via: https://github.com/npch/reporunner/blob/main/notebooks/zenodo.ipynb

#search_query = "software"
#r = requests.get("https://zenodo.org/api/records", params={'type': 'software'})  
  # this comes from the format w/in 'records' entity here: https://pythonhosted.org/zenodo/api/records.html  

This search just pulls software records from the api ^  

In [9]:
r.headers

{'server': 'nginx', 'date': 'Mon, 16 Oct 2023 15:14:32 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'x-ratelimit-limit': '60', 'x-ratelimit-remaining': '58', 'x-ratelimit-reset': '1697469332', 'retry-after': '59', 'permissions-policy': 'interest-cohort=()', 'x-frame-options': 'sameorigin', 'x-xss-protection': '1; mode=block', 'x-content-type-options': 'nosniff', 'content-security-policy': "default-src 'self' fonts.googleapis.com *.gstatic.com data: 'unsafe-inline' 'unsafe-eval' blob: zenodo-broker.web.cern.ch zenodo-broker-qa.web.cern.ch maxcdn.bootstrapcdn.com cdnjs.cloudflare.com ajax.googleapis.com webanalytics.web.cern.ch", 'strict-transport-security': 'max-age=31556926; includeSubDomains, max-age=15768000', 'referrer-policy': 'strict-origin-when-cross-origin', 'access-control-allow-origin': '*', 'access-control-expose-headers': 'Content-Type, ETag, Link, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset', 'set-cooki

In [20]:
r.url

'https://zenodo.org/api/records?type=software'